In [6]:
# install cartopy using conda
! conda install -c conda-forge cartopy --yes
! conda install -c pyviz geoviews --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.5.0



## Package Plan ##

  environment location: /Users/reneeagnor/anaconda3

  added / updated specs:
    - cartopy


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2023.05.30~ --> conda-forge::ca-certificates-2023.5.7-h8857fd0_0 
  certifi            pkgs/main/osx-64::certifi-2023.5.7-py~ --> conda-forge/noarch::certifi-2023.5.7-pyhd8ed1ab_0 
  openssl              pkgs/main::openssl-1.1.1u-hca72f7f_0 --> conda-forge::openssl-1.1.1u-h8a1eda9_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of conda exists. <==


# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key
print(geoapify_key)

0917a41455024d4b8b8ba5013510cd7b


In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../Starter_Code/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hastings,50.8552,0.5729,17.86,24,29,6.01,GB,1687737521
1,1,port-aux-francais,-49.3500,70.2167,1.98,99,100,13.48,TF,1687737521
2,2,bukama,-9.2000,25.8500,18.05,52,6,1.08,CD,1687737521
3,3,bethel,41.3712,-73.4140,26.47,77,6,1.73,US,1687737427
4,4,carupano,10.6678,-63.2585,26.38,85,14,4.33,VE,1687737521


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display
humidity = city_data_df["Humidity"].astype(int)
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
city_data_df = city_data_df.dropna()
# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
68,68,petropavlovsk-kamchatsky,53.0452,158.6483,21.33,53,0,1.00,RU,1687737482
76,76,kolonia,50.9333,6.9500,23.87,62,0,1.54,DE,1687737251
113,113,mahriz,31.5917,54.4318,25.28,26,0,3.15,IR,1687737536
120,120,bismarck,46.8083,-100.7837,25.80,52,0,3.60,US,1687737537
270,270,da nang,16.0678,108.2208,25.99,89,0,1.54,VN,1687737558
377,377,eruh,37.7517,42.1811,21.30,32,0,1.85,TR,1687737572
401,401,tobruk,32.0836,23.9764,23.45,79,0,3.71,LY,1687737575
408,408,el battan,36.8037,9.8442,23.82,76,0,1.79,TN,1687737576
461,461,ain el bya,35.8039,-0.3018,23.16,66,0,2.74,DZ,1687737582
519,519,siroki brijeg,43.3833,17.5928,23.51,87,0,3.24,BA,1687737591


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = pd.DataFrame({"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity})


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
68,petropavlovsk-kamchatsky,RU,53.0452,158.6483,53,
76,kolonia,DE,50.9333,6.9500,62,
113,mahriz,IR,31.5917,54.4318,26,
120,bismarck,US,46.8083,-100.7837,52,
270,da nang,VN,16.0678,108.2208,89,
377,eruh,TR,37.7517,42.1811,32,
401,tobruk,LY,32.0836,23.9764,79,
408,el battan,TN,36.8037,9.8442,76,
461,ain el bya,DZ,35.8039,-0.3018,66,
519,siroki brijeg,BA,43.3833,17.5928,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
kolonia - nearest hotel: Wasserturm Hotel Cologne
mahriz - nearest hotel: هتل رستوران شاه عباسی
bismarck - nearest hotel: Radisson Hotel Bismarck
da nang - nearest hotel: Thanh Thanh Hotel
eruh - nearest hotel: No hotel found
tobruk - nearest hotel: فندق زهرة البطنان -طبرق
el battan - nearest hotel: No hotel found
ain el bya - nearest hotel: فندق أريحا
siroki brijeg - nearest hotel: Hotel Park
grenada - nearest hotel: Allegro Granada


,City,Country,Lat,Lng,Humidity,Hotel Name
68,petropavlovsk-kamchatsky,RU,53.0452,158.6483,53,ООО Постоялый двор
76,kolonia,DE,50.9333,6.9500,62,Wasserturm Hotel Cologne
113,mahriz,IR,31.5917,54.4318,26,هتل رستوران شاه عباسی
120,bismarck,US,46.8083,-100.7837,52,Radisson Hotel Bismarck
270,da nang,VN,16.0678,108.2208,89,Thanh Thanh Hotel
377,eruh,TR,37.7517,42.1811,32,No hotel found
401,tobruk,LY,32.0836,23.9764,79,فندق زهرة البطنان -طبرق
408,el battan,TN,36.8037,9.8442,76,No hotel found
461,ain el bya,DZ,35.8039,-0.3018,66,فندق أريحا
519,siroki brijeg,BA,43.3833,17.5928,87,Hotel Park


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)